In [ ]:

import glob

import rasterio
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling
from shapely.geometry import LineString

import utils.visualization as viz
from utils.blobifier import blobify_raster_file
from utils.vectorizer.vector_builder import VectorBuilder

from simplification.cutil import simplify_coords

! export PYTHONOPTIMIZE="" # show asserts










In [ ]:

input_filepath = glob.glob('data/sources/*.tif')[0]
output_filepath = 'data/sources/cleaned.tif'

cleaned = blobify_raster_file(input_filepath, output_filepath, 50)

with rasterio.open(input_filepath) as src:
    meta = src.meta
    input_raster = src.read(1)

with rasterio.open(output_filepath) as src:
    meta = src.meta
    output_raster = src.read(1)

config = viz.get_show_config(input_raster)
viz.show_raster(input_raster, *config)
viz.show_raster(output_raster, *config)

In [ ]:
input_filepath = "data/sources/cleaned.tif"
output_filepath = "data/sources/smoothed.shp"

meters_per_pixel = 30
tolerance = meters_per_pixel

def simplify(segment):
    # Simplification will turn rings into what are effectively points.
    # We cut the ring in half to provide simplification with non-ring segments instead.
    if segment.is_ring:
        assert len(segment.coords) >= 3
        midpoint_idx = len(segment.coords) // 2
        segment1 = LineString(segment.coords[:midpoint_idx+1])
        segment2 = LineString(segment.coords[midpoint_idx:])
        simplified_coords1 = simplify_coords(segment1.coords, tolerance)
        simplified_coords2 = simplify_coords(segment2.coords, tolerance)
        simplified = LineString(simplified_coords1[:-1] + simplified_coords2)
    else:
        simplified_coords = simplify_coords(segment.coords, tolerance)
        simplified = LineString(simplified_coords)
    return simplified

vector_builder = VectorBuilder(input_filepath)
vector_builder.run_per_segment(simplify)
vector_builder.rebuild()
vector_builder.save(output_filepath)

data = vector_builder.data
vectors = vector_builder.areas
viz.show_raster(data, *viz.get_show_config(data))

labels = [v.label for v in vectors]
cmap = viz.generate_color_map(labels)

polygons = [v.polygon for v in vectors]
viz.show_polygons(polygons, labels, color_map=cmap)

modified_polygons = [v.modified_polygon for v in vectors]
viz.show_polygons(modified_polygons, labels, color_map=cmap)